# Pendant drop measurement check

In [ ]:
# Imports

## Packages

## Custom code
from hardware.opentrons.opentrons_api import OpentronsAPI
from hardware.opentrons.configuration import Configuration
from hardware.opentrons.droplet_manager import DropletManager
from hardware.cameras.pendant_drop_camera import PendantDropCamera
from analysis.plots import Plotter
from utils.load_save_functions import load_settings
from utils.utils import calculate_equillibrium_value
from hardware.sensor.sensor_api import SensorAPI

# Initialisation
opentrons_api = OpentronsAPI()
opentrons_api.initialise()
config = Configuration(opentrons_api=opentrons_api)
labware = config.load_labware()
containers = config.load_containers()
pipettes = config.load_pipettes()
left_pipette = pipettes["left"]
right_pipette = pipettes["right"]
pendant_drop_camera = PendantDropCamera() 
sensor = SensorAPI()
plotter = Plotter()
droplet_manager = DropletManager(
    left_pipette=left_pipette,
    containers=containers,
    pendant_drop_camera=pendant_drop_camera,
    opentrons_api=opentrons_api,
    plotter=plotter
)
settings = load_settings()


2025-08-15 16:34:45,452 - INFO - Protocol already uploaded, using existing protocol.
2025-08-15 16:34:52,604 - INFO - Run created succesfully (ID: fb1c2649-e9ad-4be2-9e2c-2d0f04afacf9).
2025-08-15 16:34:54,919 - INFO - All custom labware definitions added.
2025-08-15 16:34:55,959 - INFO - Labware loaded successfully.
2025-08-15 16:34:56,018 - INFO - Containers loaded successfully.
2025-08-15 16:35:00,223 - INFO - Pipettes loaded successfully.


[Sensor server] Sensor server started in background
[Sensor server] Connected to COM6
 * Serving Flask app 'hardware.sensor.sensor_server'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.0.73:5001
Press CTRL+C to quit
127.0.0.1 - - [15/Aug/2025 16:57:14] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 17:05:15] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 17:13:20] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 17:16:10] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 17:24:03] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 17:31:23] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 17:38:29] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 17:44:50] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 17:50:07] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 17:55:24] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 18:09:37] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 18:15:28] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 18:38:13] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2025 18:46:1

[Sensor server error] GetOverlappedResult failed (PermissionError(13, 'Access is denied.', None, 5))


127.0.0.1 - - [20/Aug/2025 15:50:13] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2025 16:02:24] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2025 16:14:31] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2025 16:26:18] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2025 16:37:02] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2025 16:46:43] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2025 16:54:42] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2025 17:00:51] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2025 17:05:08] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2025 17:08:40] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2025 17:12:05] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2025 17:16:38] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2025 17:42:29] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2025 17:54:34] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2025 18:06:32] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2025 18:18:12] "GE

In [4]:
left_pipette.pick_up_needle()

2025-08-15 16:36:10,810 - INFO - Picked up needle.


In [2]:
opentrons_api.home()

2025-08-15 16:35:19,000 - INFO - Robot homed.


## Part 0: Set well of interest

In [ ]:
well_ID_of_interest = "7H1"
droplet_manager.source = containers[well_ID_of_interest]

## Part 1: Focus camera

### Prepare pendant drop via standard method

In [ ]:
droplet_manager._prepare_pendant_drop()

### Dispense pendant drop manually
repeat this block to dispense more

In [6]:
left_pipette.dispense(
    volume=0,
    destination=containers['drop_stage'],
    depth_offset=settings["PENDANT_DROP_DEPTH_OFFSET"]+0.5,
    flow_rate=1,
    log=False,
    update_info=False
)

### Steps to focus camera
- go to [`check_camera.py`](check_camera.py)
- run script, a video stream of the pendant drop should occur
- manually focus the camera
- press q to exit

### Return pendant drop

In [ ]:
volume_dispensed = 17 - left_pipette.volume # prepare pendant drop aspirate 17 uL of source (p20 - 3 uL air gap)
droplet_manager._return_pendant_drop(
    drop_volume=volume_dispensed
)

## Part 2: Check measurement

In [ ]:
well_ID_of_interest = "7B2"

In [ ]:
dynamic_surface_tension, drop_volume, drop_count = droplet_manager.measure_pendant_drop(
    source=containers[well_ID_of_interest],
    max_measure_time=30 #s
)
st_eq = calculate_equillibrium_value(
    x=dynamic_surface_tension,
    n_eq_points=100,
    column_index=1,
)

print(f"equillibrium st: {st_eq} mN/m")
meta_data = sensor.capture_sensor_data()
print(f"temperature: {meta_data["Temperature (C)"]}")
print(f"humidity: {meta_data["Humidity (%)"]}")
